In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from math import sin, cos
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error as mse

np.random.seed(42)

In [97]:
df = pd.read_csv('../cleaned_data_mp.csv', index_col=0)
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,pitch,yaw,roll
image00002,0.508591,0.239336,0.430168,0.252891,0.187046,0.383993,0.385302,0.306882,-0.399231,0.018227,0.085676
image00004,0.897473,0.463788,0.510713,0.311286,0.069614,0.529695,0.340960,0.559336,0.470065,1.189533,0.300959
image00008,0.604275,0.391871,0.264828,0.376344,0.250255,0.383493,0.438322,0.460450,-0.175379,0.299208,-0.373374
image00013,0.512615,0.467233,0.393340,0.199575,0.178440,0.432988,0.418808,0.567076,-0.026812,0.011965,-0.220662
image00014,0.561266,0.465798,0.482575,0.155688,0.183200,0.444852,0.445997,0.547594,0.057119,0.110732,-0.043283


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1835 entries, image00002 to image04375
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      1835 non-null   float64
 1   x2      1835 non-null   float64
 2   x3      1835 non-null   float64
 3   x4      1835 non-null   float64
 4   x5      1835 non-null   float64
 5   x6      1835 non-null   float64
 6   x7      1835 non-null   float64
 7   x8      1835 non-null   float64
 8   pitch   1835 non-null   float64
 9   yaw     1835 non-null   float64
 10  roll    1835 non-null   float64
dtypes: float64(11)
memory usage: 172.0+ KB


In [111]:
# some util functions
def split_x_y(data):
    return data[:, :-3], data[:, -3:]

def split_train_dev_x_y(df, train_size=0.9):
    data_train, data_test = train_test_split(df, train_size=train_size)
    X_train, Y_train = split_x_y(data_train.values)
    X_dev, Y_dev = split_x_y(data_test.values)
    return X_train, Y_train, X_dev, Y_dev 

def draw_axis(img, yaw, pitch, roll, tdx=None, tdy=None, size = 100):

    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img
    
# def show_image(path):
#     capture = cv2.VideoCapture(path)
#     ret, frame = capture.read()
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # convert to RGB format for matplotlib
#     landmarks, pitch, yaw, roll = read_mat(path)
#     x,y = landmarks[0,32], landmarks[1,32]
#     plt.imshow(draw_axis(frame,-yaw, pitch, roll, x, y))
#     plt.show()
#     capture.release()
#     cv2.destroyAllWindows()
    
def try_model(X_train, Y_train, X_dev, Y_dev, model):
    x_model = model.fit(X_train, Y_train)

    print('\nAccuracy on training set: ')
    h = x_model.predict(X_train)
    # print('model r2_score =', r2_score(Y_train , h))
    print('model MSE =', mse(Y_train , h))

    print('\nAccuracy on dev set: ')
    h = x_model.predict(X_dev)
    print('model MSE =', mse(Y_dev, h))
    


def try_models(X_train, Y_train, X_dev, Y_dev, model):
    pitch_model = model().fit(X_train, Y_train[:, 0])
    yaw_model = model().fit(X_train, Y_train[:, 1])
    roll_model = model().fit(X_train, Y_train[:, 2])

    print('\nAccuracy on training set: ')
    h_pitch = pitch_model.predict(X_train)
    h_yaw = yaw_model.predict(X_train)
    h_roll = roll_model.predict(X_train)
    print('Pitch r2_score =', r2_score(Y_train[:, 0] , h_pitch))
    print('Yaw r2_score =', r2_score(Y_train[:, 1] , h_yaw))
    print('Roll r2_score =', r2_score(Y_train[:, 2] , h_roll))

    print('\nAccuracy on dev set: ')
    h_pitch = pitch_model.predict(X_dev)
    h_yaw = yaw_model.predict(X_dev)
    h_roll = roll_model.predict(X_dev)
    print('Pitch r2_score =', r2_score(Y_dev[:, 0] , h_pitch))
    print('Yaw r2_score =', r2_score(Y_dev[:, 1] , h_yaw))
    print('Roll r2_score =', r2_score(Y_dev[:, 2] , h_roll))


In [100]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Try LinearRegression on 21pts df.
X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df, train_size=0.8)
try_models(X_train, Y_train, X_dev, Y_dev, LinearRegression)


Accuracy on training set: 
Pitch r2_score = 0.14461637715242082
Yaw r2_score = 0.8165393589990336
Roll r2_score = 0.29079946985660476

Accuracy on dev set: 
Pitch r2_score = 0.5759115337505021
Yaw r2_score = 0.8782557681210683
Roll r2_score = 0.5777432133769598


In [114]:
# try_model(X_train, Y_train[:, 0], X_dev, Y_dev[:, 0], model=SVR(C=100))
# try_model(X_train, Y_train[:, 1], X_dev, Y_dev[:, 1], model=SVR(C=100))
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=SVR(C=100))


Accuracy on training set: 
model MSE = 0.021009019505625778

Accuracy on dev set: 
model MSE = 0.046285005618846325


In [108]:
# try_model(X_train, Y_train[:, 0], X_dev, Y_dev[:, 0], model=RandomForestRegressor(n_estimators=500, max_depth=6, min_samples_leaf=4))
# try_model(X_train, Y_train[:, 1], X_dev, Y_dev[:, 1], model=RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=4))
try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=RandomForestRegressor(n_estimators=500, max_depth=6, min_samples_leaf=4))


Accuracy on training set: 
model r2_score = 0.5353536038456876

Accuracy on dev set: 
model r2_score = -0.36202281493402544


In [103]:
from xgboost import XGBRegressor

try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2],
model=XGBRegressor(n_estimators=100, nthread=4))


Accuracy on training set: 
model r2_score = 0.9996995218126943

Accuracy on dev set: 
model r2_score = -5.580754316223929


In [109]:
# try_model(X_train, Y_train[:, 0], X_dev, Y_dev[:, 0], model=XGBRegressor(n_estimators=5000, nthread=4))
# try_model(X_train, Y_train[:, 1], X_dev, Y_dev[:, 1], model=XGBRegressor(n_estimators=50, nthread=4))
# try_model(X_train, Y_train[:, 2], X_dev, Y_dev[:, 2], model=XGBRegressor(n_estimators=50, nthread=4))


Accuracy on training set: 
model r2_score = 0.9999948929962871

Accuracy on dev set: 
model r2_score = -7.189166913332533
